In [1]:
from pathlib import Path
import sys
from tree_format_A2B import main as a2b_convert
from tree_format_B2A import main as b2a_convert
from tree_converter import TreeConverter
import os

# 格式常量定义
FORMAT_A = 'A'
FORMAT_B = 'B'
FORMAT_C = 'C'

def detect_format(content):
    """检测文件格式"""
    if '├──' in content:
        return FORMAT_B
    elif '┣' in content:
        return FORMAT_C
    return FORMAT_A

def convert_file(input_file, output_file, current_format, target_format):
    """执行具体的转换操作"""
    temp_file = 'temp_b.txt'
    try:
        if current_format == FORMAT_A:
            # A格式首先转换为B格式
            sys.argv = [sys.argv[0], '-i', input_file, '-o', temp_file]
            a2b_convert()
            
            if target_format == FORMAT_B:
                # 如果目标是B格式，直接移动临时文件到目标位置
                Path(temp_file).replace(output_file)
            else:  # 目标是C格式
                # 读取B格式内容并转换为C格式
                b_content = Path(temp_file).read_text(encoding='utf-8')
                c_content = TreeConverter.convert_format(b_content, FORMAT_C)
                Path(output_file).write_text(c_content, encoding='utf-8')
                
        elif current_format == FORMAT_B:
            if target_format == FORMAT_A:
                sys.argv = [sys.argv[0], '-i', input_file, '-o', output_file]
                b2a_convert()
            else:  # 转换为C格式
                content = Path(input_file).read_text(encoding='utf-8')
                result = TreeConverter.convert_format(content, FORMAT_C)
                Path(output_file).write_text(result, encoding='utf-8')
                
        else:  # 当前是C格式
            # C格式首先转换为B格式
            content = Path(input_file).read_text(encoding='utf-8')
            b_content = TreeConverter.convert_format(content, FORMAT_B)
            Path(temp_file).write_text(b_content, encoding='utf-8')
            
            if target_format == FORMAT_A:
                # 从B格式转换为A格式
                sys.argv = [sys.argv[0], '-i', temp_file, '-o', output_file]
                b2a_convert()
            else:  # 目标是B格式
                Path(temp_file).replace(output_file)
    
    finally:
        # 清理临时文件
        if os.path.exists(temp_file):
            try:
                Path(temp_file).unlink()
            except:
                pass

def main():
    # 直接设置输入输出参数
    input_file = "C.txt"          # 输入文件名
    output_file = "output.txt" # 输出文件名
    target_format = "A"               # 目标格式：A、B或C
     
    try:
        # 读取文件内容
        content = Path(input_file).read_text(encoding='utf-8')
        # 检测当前格式
        current_format = detect_format(content)
        
        if current_format == target_format:
            print(f"文件已经是{current_format}格式，无需转换")
            return
            
        # 执行转换
        convert_file(input_file, output_file, current_format, target_format)
        print(f"转换完成！已将{current_format}格式转换为{target_format}格式")
        print(f"结果保存至：{output_file}")
            
    except Exception as e:
        print(f"错误：{str(e)}")
        sys.exit(1)

if __name__ == '__main__':
    main()


缩进文本已保存到: output.txt
转换完成！已将C格式转换为A格式
结果保存至：output.txt
